<a href="https://colab.research.google.com/github/purrchinelearning/basic-pytorch/blob/main/Yolov1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import sys
print(sys.version)

3.7.12 (default, Sep 10 2021, 00:21:48) 
[GCC 7.5.0]


In [4]:
import torch
print("Torch version:{}".format(torch.__version__))
print("cuda version: {}".format(torch.version.cuda))
print("cudnn version:{}".format(torch.backends.cudnn.version()))

Torch version:1.9.0+cu111
cuda version: 11.1
cudnn version:8005


In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torchsummary import summary
from torchvision import transforms,datasets
from torch.utils.data import TensorDataset, DataLoader, Dataset
import cv2
from PIL import Image
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [12]:
A_C = [
  (7,64,2,3), "M",
  (3,192,1,1), "M",
  (1,128,1,0), (3,256,1,1),(1,256,1,0),(3,512,1,1),"M",
  [(1,256,1,0),(3,512,1,1),4],(1,512,1,0),(3,1024,1,1),"M",
  [(1,512,1,0),(3,1024,1,1),2],(3,1024,1,1),(3,1024,2,1),
  (3,1024,1,1),(3,1024,1,1)
]

In [73]:
class ConvBlock(nn.Module):
  def __init__(self,in_ch,out_ch,k,s,p):
    super(ConvBlock,self).__init__()
    self.conv = nn.Conv2d(in_ch,out_ch,k,s,p,bias=False)
    self.bn = nn.BatchNorm2d(out_ch)
    self.LR = nn.LeakyReLU(0.1)

  def forward(self,x):
    return self.LR(self.bn(self.conv(x)))
  
class Yolov1(nn.Module):
  def __init__(self,in_ch1=3,S=[7,2,20]):
    super(Yolov1,self).__init__()
    self.S = S
    self.arc = A_C
    self.S = S[0]
    self.B = S[1]
    self.C = S[2]
    self.fcs = self._CF(self.S,self.B,self.C)
    self.in_ch1 = in_ch1
    self.DN = self._CCL(self.arc,self.in_ch1)

  def _CCL(self, arc, in_ch1):
    layers=[]
    c = self.in_ch1
    for i in self.arc:
      if i=="M":
        layers.append(nn.MaxPool2d(2,2))
      else:
        if type(i) == list:
          in_1 = i[0]
          in_2 = i[1]
          for j in range(i[2]):
            layers += [ConvBlock(c,in_1[1],in_1[0],in_1[2],in_1[3])]
            layers.append(ConvBlock(in_1[1],in_2[1],in_2[0],in_2[2],in_2[3]))
            c = in_2[1]
        else:
          layers += [ConvBlock(c,i[1],i[0],i[2],i[3])]
          c = i[1]
    return nn.Sequential(*layers)
  
  def _CF(self,S,B,C):
    S, B, C = self.S, self.B, self.C
    return nn.Sequential(
        nn.Flatten(),
        nn.Linear(1024*S*S,4096),
        nn.LeakyReLU(0.1),
        nn.Linear(4096,S*S*(B*5+C))
    )

  def forward(self,x):
    x = self.DN(x)
    return self.fcs(torch.flatten(x,start_dim=1))

model = Yolov1().to(device)
summary(model,(3,448,448),device = device.type)

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           9,408
       BatchNorm2d-2         [-1, 64, 224, 224]             128
         LeakyReLU-3         [-1, 64, 224, 224]               0
         ConvBlock-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 192, 112, 112]         110,592
       BatchNorm2d-7        [-1, 192, 112, 112]             384
         LeakyReLU-8        [-1, 192, 112, 112]               0
         ConvBlock-9        [-1, 192, 112, 112]               0
        MaxPool2d-10          [-1, 192, 56, 56]               0
           Conv2d-11          [-1, 128, 56, 56]          24,576
      BatchNorm2d-12          [-1, 128, 56, 56]             256
        LeakyReLU-13          [-1, 128, 56, 56]               0
        ConvBlock-14          [-1, 128,